In [49]:
start_date = "2020-01-01"
end_date = "2025-10-31"

NSE = [
    'ARM',      # ARM Cement
    'ABSA',     # Absa Bank Kenya
    'BAMB',     # Bamburi Cement
    'BOC',      # B O C Kenya
    'BRIT',     # Britam (Kenya)
    'CARB',     # Carbacid Investments
    'CTUM',     # Centum Investment
    'SBIC',     # Stanbic Holdings
    'LBTY',     # Liberty Kenya Holdings
    'COOP',     # Co-operative Bank of Kenya
    'CRWN',     # Crown Paints Kenya
    'CGEN',     # Car & General (K)
    'OCH',      # Olympia Capital Holdings
    'DTK',      # Diamond Trust Bank Kenya
    'EABL',     # East African Breweries
    'CABL',     # East African Cables
    'EGAD',     # Eaagads
    'EQTY',     # Equity Group Holdings
    'EVRD',     # Eveready East Africa
    'XPRS',     # Express Kenya
    'WTK',      # Williamson Tea Kenya
    'HAFR',     # Home Afrika
    'HFCK',     # HF Group
    'IMH',      # I&M Holdings
    'JUB',      # Jubilee Holdings
    'KEGN',     # KenGen Company
    'KUKZ',     # Kakuzi
    'KQ',       # Kenya Airways
    'KCB',      # KCB Group
    'KNRE',     # Kenya Re-Insurance Corporation
    'KPLC',     # Kenya Power & Lighting
    'KAPC',     # Kapchorua Tea Kenya
    'KURV',     # Kurwitu Ventures
    'LKL',      # Longhorn Publishers
    'LIMT',     # Limuru Tea
    'MSC',      # Mumias Sugar Co
    'NCBA',     # NCBA Group
    'NMG',      # Nation Media Group
    'NSE',      # Nairobi Securities Exchange
    'SLAM',     # Sanlam Kenya
    'SCOM',     # Safaricom
    'SMER',     # Sameer Africa
    'SCAN',     # WPP Scangroup
    'SCBK',     # Standard Chartered Bank Kenya
    'SASN',     # Sasini
    'SGL',      # Standard Group
    'TCL',      # TransCentury
    'TPSE',     # TPS Eastern Africa
    'UCHM',     # Uchumi Supermarkets
    'UNGA',     # Unga Group
    'TOTL',     # TotalEnergies Marketing Kenya
    'UMME',     # Umeme
    'FTGH',     # Flame Tree Group Holdings
    'NBV',      # Nairobi Business Ventures
    'DCON',     # Deacons (East Africa)
    'BKG',      # BK Group
    'SMWF',     # Satrix MSCI World Feeder ETF
    'SKL',      # Shri Krishana Overseas
]

In [50]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
from tqdm import tqdm
import time
import warnings

warnings.filterwarnings("ignore")

username = "crismbici"
password = "990306@CrisMbici"

tv = TvDatafeed(username, password)

all_data = []
nan_count = 0

for symbol in tqdm(NSE, desc="Fetching NSE data"):
    try:
        data = tv.get_hist(
            symbol=symbol,
            exchange="NSEKE",
            interval=Interval.in_daily,
            n_bars=5000
        )

        if data is not None and not data.empty:
            data = data.reset_index()
            data["ticker"] = symbol
            nan_count += data.isna().sum().sum()
            all_data.append(data)
        time.sleep(0.2)

    except Exception:
        continue

if all_data:
    df_all = pd.concat(all_data, ignore_index=True)
    df_all["datetime"] = pd.to_datetime(df_all["datetime"])
    df_all = df_all[(df_all["datetime"] >= start_date) & (df_all["datetime"] <= end_date)]
    df_all.sort_values(["ticker", "datetime"], inplace=True)
    df_all.dropna(subset=["open", "high", "low", "close"], inplace=True)
    df_all.to_csv("start.csv", index=False)
    print(f"Data saved to start.csv ({nan_count} missing values found before cleaning)")
else:
    print("No data fetched. Check connection or symbols.")

Fetching NSE data: 100%|██████████| 58/58 [02:42<00:00,  2.80s/it]


Data saved to start.csv (0 missing values found before cleaning)


In [51]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# Read daily data
df = pd.read_csv("start.csv")
df["datetime"] = pd.to_datetime(df["datetime"])
df["month"] = df["datetime"].dt.to_period("M")

# Aggregate to monthly
monthly = (
    df.groupby(["ticker", "month"]).agg(
        open=("open", "first"),
        high=("high", "max"),
        low=("low", "min"),
        close=("close", "last"),
        volume=("volume", "sum")
    ).reset_index()
)

# Ensure month is a timestamp for sorting and merging later
monthly["month"] = monthly["month"].dt.to_timestamp()

# Sort for rolling calculations
monthly = monthly.sort_values(["ticker", "month"]).reset_index(drop=True)

# Feature engineering
def compute_features(group):
    g = group.copy()
    g["return_1m"] = g["close"].pct_change()
    g["log_return_1m"] = np.log(g["close"] / g["close"].shift(1))

    # Multi-period returns
    for n in [3, 6, 12]:
        g[f"return_{n}m"] = g["close"].pct_change(n)
        g[f"log_return_{n}m"] = np.log(g["close"] / g["close"].shift(n))

    # Volatility of log returns
    for n in [3, 6, 12]:
        g[f"vol_{n}m"] = g["log_return_1m"].rolling(n).std()

    # True Range and ATRs
    g["tr"] = np.maximum.reduce([
        g["high"] - g["low"],
        (g["high"] - g["close"].shift(1)).abs(),
        (g["low"] - g["close"].shift(1)).abs()
    ])
    g["atr"] = g["tr"]
    for n in [3, 6, 12]:
        g[f"atr_{n}m"] = g["tr"].rolling(n).mean()

    # Moving averages and ratios
    for n in [3, 6, 12]:
        g[f"sma_{n}m"] = g["close"].rolling(n).mean()
        g[f"close_sma_ratio_{n}m"] = g["close"] / g[f"sma_{n}m"]

    # High-low ratio
    g["high_low_ratio"] = g["high"] / g["low"]

    # Drawdowns
    for n in [3, 6, 12]:
        g[f"drawdown_{n}m"] = (g["close"] / g["close"].rolling(n, min_periods=1).max()) - 1

    # Rate of change
    for n in [3, 6, 12]:
        g[f"roc_{n}m"] = (g["close"] - g["close"].shift(n)) / g["close"].shift(n)

    # Positive/negative streaks
    g["positive_streak"] = (g["return_1m"] > 0).astype(int).groupby((g["return_1m"] <= 0).astype(int).cumsum()).cumsum()
    g["negative_streak"] = (g["return_1m"] < 0).astype(int).groupby((g["return_1m"] >= 0).astype(int).cumsum()).cumsum()

    # Skewness and kurtosis of log returns
    for n in [3, 6, 12]:
        g[f"skew_{n}m"] = g["log_return_1m"].rolling(n).apply(lambda x: skew(x, bias=False) if len(x.dropna()) > 1 else np.nan, raw=False)
        g[f"kurtosis_{n}m"] = g["log_return_1m"].rolling(n).apply(lambda x: kurtosis(x, bias=False) if len(x.dropna()) > 1 else np.nan, raw=False)

    # Target: direction (1 if return positive, else 0)
    g["direction"] = (g["return_1m"] > 0).astype(int)

    return g

monthly_features = monthly.groupby("ticker", group_keys=False).apply(compute_features)

# Drop unnecessary columns
monthly_features.drop(columns=["tr"], inplace=True, errors="ignore")

# Save result
monthly_features.to_csv("step2_stock_features.csv", index=False)
print("Saved to step2_stock_features.csv")

Saved to step2_stock_features.csv


In [52]:
NSE_BY_SECTOR = {
    'AGRICULTURAL': [
        'EGAD',      # Eaagads Ltd
        'KAPC',      # Kapchorua Tea Kenya Plc
        'KUKZ',      # Kakuzi Plc
        'LIMT',      # Limuru Tea Co. Ltd
        'SASN',      # Sasini Plc
        'WTK',       # Williamson Tea Kenya Plc
    ],
    
    'AUTOMOBILES_AND_ACCESSORIES': [
        'CGEN',      # Car & General (K) Ltd
    ],
    
    'BANKING': [
        'ABSA',      # ABSA Bank Kenya Plc
        'SBIC',      # Stanbic Holdings Ltd
        'IMH',       # I & M Holdings Plc
        'DTK',       # Diamond Trust Bank Kenya Ltd
        'HFCK',      # HF Group Plc
        'KCB',       # KCB Group Plc
        'NCBA',      # NCBA Group Plc
        'SCBK',      # Standard Chartered Bank Kenya Ltd
        'EQTY',      # Equity Group Holdings Plc
        'COOP',      # The Co-operative Bank of Kenya Ltd
        'BKG',       # BK Group Plc
    ],
    
    'COMMERCIAL_AND_SERVICES': [
        'XPRS',      # Express Kenya Plc
        'KQ',        # Kenya Airways Ltd
        'NMG',       # Nation Media Group Plc
        'SGL',       # Standard Group Plc
        'TPSE',      # TPS Eastern Africa (Serena) Ltd
        'SCAN',      # WPP Scangroup Plc
        'UCHM',      # Uchumi Supermarket Plc
        'EVRD',      # Eveready East Africa Ltd
        'LKL',       # Longhorn Publishers Plc
        'DCON',      # Deacons (East Africa) Plc
        'SMER',      # Sameer Africa Plc
        'NBV',       # Nairobi Business Ventures Ltd
        'HEL',       # Homeboyz Entertainment Plc (Not in your list but listed on NSE)
    ],
    
    'CONSTRUCTION_AND_ALLIED': [
        'ARM',       # ARM Cement Plc
        'BAMB',      # Bamburi Cement Ltd
        'CRWN',      # Crown Paints Kenya Plc
        'CABL',      # E.A Cables Ltd (Not in your list but listed on NSE)
        'EAPC',      # E.A Portland Cement Ltd (Not in your list but listed on NSE)
    ],
    
    'ENERGY_AND_PETROLEUM': [
        'TOTL',      # Total Kenya Ltd
        'KEGN',      # KenGen Plc
        'KPLC',      # Kenya Power & Lighting Plc
        'UMME',      # Umeme Ltd
    ],
    
    'INSURANCE': [
        'JUB',       # Jubilee Holdings Ltd
        'SLAM',      # Sanlam Kenya Plc
        'KNRE',      # Kenya Re-Insurance Corporation Ltd
        'LBTY',      # Liberty Kenya Holdings
        'BRIT',      # Britam Holdings Plc
        'CIC',       # CIC Insurance Group Ltd
    ],
    
    'INVESTMENT': [
        'OCH',       # Olympia Capital Holdings Ltd
        'CTUM',      # Centum Investment Plc
        'TCL',       # Trans-Century Plc
        'HAFR',      # Home Afrika Ltd
        'KURV',      # Kurwitu Ventures Ltd
        'NSE',       # Nairobi Securities Exchange Plc
    ],
    
    'MANUFACTURING': [
        'BOC',       # B.O.C Kenya Plc
        'BATK',      # British American Tobacco Kenya Plc (Not in your list but listed on NSE)
        'CARB',      # Carbacid Investments Plc
        'EABL',      # East African Breweries Ltd
        'MSC',       # Mumias Sugar Co. Ltd
        'UNGA',      # Unga Group Ltd
        'ORCH',      # Kenya Orchards Ltd (Not in your list but listed on NSE)
        'FTGH',      # Flame Tree Group Holdings Ltd
        'SKL',       # Shri Krishana Overseas Plc (packaging/manufacturing)
    ],
    
    'TELECOMMUNICATION': [
        'SCOM',      # Safaricom Plc
    ],
    
    'EXCHANGE_TRADED_FUNDS': [
        'SMWF',      # Satrix MSCI World Feeder ETF
        'GLD',       # ABSA New Gold ETF (Not in your list but listed on NSE)
    ],
}

In [53]:
import pandas as pd
import numpy as np

# Read stock-level features
df = pd.read_csv("step2_stock_features.csv")
df["month"] = pd.to_datetime(df["month"])

# Build mapping from ticker to industry
ticker_to_industry = {
    ticker: sector
    for sector, tickers in NSE_BY_SECTOR.items()
    for ticker in tickers
}

# Assign industry
df["industry"] = df["ticker"].map(ticker_to_industry)

# Check for missing industries
missing_tickers = df.loc[df["industry"].isna(), "ticker"].unique().tolist()

# Compute industry-level metrics
industry_features = (
    df.groupby(["industry", "month"]).agg(
        industry_return_mean=("return_1m", "mean"),
        industry_return_median=("return_1m", "median"),
        industry_total_volume=("volume", "sum"),
        industry_avg_price=("close", "mean"),
        industry_volatility=("log_return_1m", "std"),
        industry_growth_rate=("return_3m", "mean")
    ).reset_index()
)

# Rolling smoothed metrics (3-month)
industry_features = industry_features.sort_values(["industry", "month"])
for col, new_col in [
    ("industry_return_mean", "industry_return_mean_smooth"),
    ("industry_growth_rate", "industry_growth_rate_smooth"),
    ("industry_volatility", "industry_volatility_smooth")
]:
    industry_features[new_col] = (
        industry_features.groupby("industry")[col]
        .transform(lambda x: x.rolling(3, min_periods=1).mean())
    )

# Ranks within month (across industries)
rank_cols = [
    "industry_return_mean",
    "industry_growth_rate",
    "industry_total_volume",
    "industry_volatility"
]
for col in rank_cols:
    industry_features[f"{col}_rank"] = (
        industry_features.groupby("month")[col]
        .rank(ascending=False, method="dense")
    )

# Optional composite score
industry_features["industry_score_opt"] = (
    0.4 * industry_features["industry_return_mean"] +
    0.3 * industry_features["industry_growth_rate"] -
    0.3 * industry_features["industry_volatility"]
)

# Merge back to stock-level
merged = df.merge(industry_features, on=["industry", "month"], how="left")

# Save
merged.to_csv("step3_industry_features.csv", index=False)
print("Saved to step3_industry_features.csv")

# Report missing industries
if missing_tickers:
    print("Tickers missing industry assignment:", missing_tickers)
else:
    print("All tickers successfully assigned an industry.")

Saved to step3_industry_features.csv
All tickers successfully assigned an industry.


In [54]:
import pandas as pd

# Read files
df = pd.read_csv("step3_industry_features.csv")
macro = pd.read_csv("macroeconomic.csv")

# Convert month and year
df["month"] = pd.to_datetime(df["month"])
macro["month"] = pd.to_datetime(
    macro["Year"].astype(int).astype(str) + "-" + macro["Month"].astype(int).astype(str) + "-01"
)

# Drop old year/month columns to avoid duplicates
macro = macro.drop(columns=["Year", "Month"], errors="ignore")

# Merge on month
merged = df.merge(macro, on="month", how="left")

# Save
merged.to_csv("step4_macro_merged.csv", index=False)
print("Saved to step4_macro_merged.csv")

# Check if any unmatched months
missing_months = df["month"][~df["month"].isin(macro["month"])].unique()
if len(missing_months):
    print("Months missing macro data:", missing_months)
else:
    print("All months matched with macro data.")

Saved to step4_macro_merged.csv
All months matched with macro data.


In [55]:
import pandas as pd

# Load dataset
df = pd.read_csv("step4_macro_merged.csv")

# Calculate % of NaNs per column
nan_percent = df.isna().mean() * 100

# Show full output without truncation
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.float_format', '{:.2f}'.format)  # Optional: format floats nicely

print("Percentage of missing values per column:")
print(nan_percent.sort_values(ascending=False))

# Reset display options to default if you want
pd.reset_option('display.max_rows')
pd.reset_option('display.float_format')

Percentage of missing values per column:
return_12m                    17.75
kurtosis_12m                  17.75
skew_12m                      17.75
roc_12m                       17.75
atr_12m                       17.75
vol_12m                       17.75
log_return_12m                17.75
sma_12m                       16.29
close_sma_ratio_12m           16.29
Unemployment                  14.32
gfcf                          14.32
return_6m                      8.99
log_return_6m                  8.99
kurtosis_6m                    8.99
vol_6m                         8.99
skew_6m                        8.99
atr_6m                         8.99
roc_6m                         8.99
industry_score_opt             8.31
sma_6m                         7.53
close_sma_ratio_6m             7.53
gdp_construction               5.70
gdp_growth_rate                5.70
gdp_manufacturing              5.70
gdp_agriculture                5.70
gdp_services                   5.70
industry_volatility_ran

In [56]:
import pandas as pd
from tqdm import tqdm

# Load your merged data
df = pd.read_csv("step4_macro_merged.csv")

# Ensure month is datetime
df['month'] = pd.to_datetime(df['month'])

# Identify columns
id_cols = ['ticker', 'industry', 'month']
target_col = 'direction'
numeric_cols = df.select_dtypes(include='number').columns.tolist()

# Remove target from numeric columns to avoid lagging it
if target_col in numeric_cols:
    numeric_cols.remove(target_col)

# Apply lag per ticker
df_lagged = []
for ticker, group in tqdm(df.groupby('ticker'), desc="Lagging features"):
    group = group.sort_values('month')
    group[numeric_cols] = group[numeric_cols].shift(1)
    df_lagged.append(group)

df_lagged = pd.concat(df_lagged, ignore_index=True)

# --- Fill NaNs with training period medians (2019-2023) ---
train_mask = (df_lagged['month'] >= start_date) & (df_lagged['month'] <= end_date) # by Nes
medians = df_lagged.loc[train_mask, numeric_cols].median()

# Fill all NaNs using these medians
df_lagged[numeric_cols] = df_lagged[numeric_cols].fillna(medians)

# Save the lagged dataset
df_lagged.to_csv("step5_lagged_data.csv", index=False)
print("Lagging complete with median fill. Saved as step5_lagged_data.csv")

Lagging features: 100%|██████████| 55/55 [00:01<00:00, 34.31it/s]


Lagging complete with median fill. Saved as step5_lagged_data.csv
